In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
from include import *
from PIL import Image
import PIL
import pywt
import numpy as np
import torch
import torchvision
from torch.autograd import Variable
import time



GPU = True
if GPU == True:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    dtype = torch.cuda.FloatTensor
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print("num GPUs",torch.cuda.device_count())
    device = 'cuda'
    if torch.cuda.device_count()==0:
        dtype = torch.FloatTensor
        device = 'cpu'
else:
    dtype = torch.FloatTensor
    device = 'cpu'


# 1. Load a test image from a dataset (now : CelebA 128x128)

In [ ]:
#dataset = 'mnist' # 'mnist' or 'celeba'
dataset = 'celeba'
path = './test_data/' + dataset + '/' 
img_name = dataset + '1' # 1-5 (for celeba), 1-6 (for mnist)
img_path = path + img_name + ".jpg"
img_pil = Image.open(img_path)
if dataset == 'celeba':
    #img_pil = img_pil.crop((60,80+20,60+64,80+84)) #crop to 3 x 64 x 64
    cx=89
    cy=121
    img_pil = img_pil.crop((cx-64, cy - 64, cx + 64, cy+64))

img_np = pil_to_np(img_pil)
print('Dimensions of input image:', img_np.shape)
img_np = img_np / np.max(img_np)


img_np_orig = 1*img_np

if dataset == 'celeba':
    plt.imshow(img_np.transpose(1,2,0))
else:
    plt.imshow(img_np[0,:,:])
    plt.gray()
plt.axis('off')
img_var = np_to_var(img_np).type(dtype)
d = img_np.shape[1]
out_ch = img_np.shape[0]
d_image = img_np.size

# normalize the pixels to [-1,1]
img_var = 2*img_var -1



# 2. Compressed sensing using generative models

## 2.1. Load a pretrained generative model on the dataset (now: PGGAN)

In [ ]:
use_gpu = True if torch.cuda.is_available() else False

# trained on high-quality celebrity faces "celebA" dataset
# this model outputs 512 x 512 pixel images
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
                       'PGAN', model_name='celeba',
                       pretrained=True, useGPU=use_gpu)
# this model outputs 256 x 256 pixel images
# model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
#                        'PGAN', model_name='celebAHQ-256',
#                        pretrained=True, useGPU=use_gpu)
G = model.netG
G.eval()

mse = torch.nn.MSELoss()


## 2.2. A=I

In [ ]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.1, lr_decay_epoch=100, factor=0.8):
    """Decay learning rate by a factor of 0.5 every lr_decay_epoch epochs."""
    lr = init_lr * (factor**(epoch // lr_decay_epoch))
    if epoch % lr_decay_epoch == 0:
        print('\nLR is set to {}'.format(lr))
        print('\n')
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

In [ ]:
print(torch.min(img_var), torch.max(img_var))

In [ ]:
#z_hat = fit(G = G, latentDim = model.config.noiseVectorDim, y= img_var, num_iter = 1600, lr_decay_epoch = 400)    
z_hat, mse_wrt_truth = CSGM(G=G, 
             latentDim = model.config.noiseVectorDim, 
             y=img_var, 
             x=img_var,
             device = device,
             num_iter=1600)
gen_image = G(z_hat)

grid = torchvision.utils.make_grid(gen_image.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

plt.show()


In [ ]:
plt.xlabel('optimizer iteration')
plt.ylabel('recovery error')
plt.semilogy(mse_wrt_truth)

## 2.3. A= Gaussian 

In [ ]:
f = 0.2 #compression rate
print('Compression rate is ', f)
m_image = int(f*d_image)
print('Number of measurements is ',m_image, ' for signal of length ', d_image)

# random Gaussian measurement process

A = torch.randn(m_image, d_image).to(device)
x = img_var.to(device).reshape(d_image)
y = torch.matmul(A,x).to(device)

#latentDim = model.config.noiseVectorDim
print(A.shape, x.shape, y.shape)

In [ ]:
t0= time.time()
z_hat, mse_wrt_loss = CSGM2(G, latentDim, y, x, A, device, num_iter = 1600)
x_hat = G(z_hat)

#x_hat.reshape(x.size(0),x.size(1))
grid = torchvision.utils.make_grid(x_hat.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

t1= time.time()
print('\ntime elapsed:',t1-t0)

# 3. Compressed Sensing using Deep decoder

## 3.1. Define the network

In [ ]:
#use decoder architecture or DC GAN architecture
decodetype = 'upsample' # transposeconv / upsample

num_channels = [120,40,20,15,10] 

output_depth = img_np.shape[0] # number of output channels
net = autoencodernet(num_output_channels=output_depth,num_channels_up=num_channels,need_sigmoid=True, 
                        decodetype=decodetype
                        ).type(dtype)

print("number of parameters: ", num_param(net))
if decodetype == 'upsample':
    print(net.decoder)
elif decodetype == 'transposeconv':
    print(net.convdecoder)
net_in = copy.deepcopy(net)



## 3.2. CS using untrained network

In [ ]:
def fit3(net, num_channels, d_image, y, A, device, num_iter = 12000, lr_decay_epoch = 3000):
    # compute the size of (fixed) latent vector and draw it uniformly  
    totalupsample = 2**(len(num_channels)-1)
    w = np.sqrt(int(d_image/3)) # =d_image / out_channels = số chiều của mỗi cạnh ảnh
    width = int(w/(totalupsample))
    height = int(w/(totalupsample))

    shape = [1,num_channels[0], width, height]  
    print("shape of latent code B1: ", shape)

    print("initializing latent code B1...")
    net_input = Variable(torch.zeros(shape))
    net_input.data.uniform_()
    net_input.data *= 1./10

    net_input_saved = net_input.data.clone()
    noise = net_input.data.clone()

    #x_in = net(net_input.type(dtype)).data.clone() #initializing image

    # processing optimization
    if decodetype=='upsample':
        p = [x for x in net.decoder.parameters() ] #list of all weigths
    elif decodetype=='transposeconv':
        p = [x for x in net.convdecoder.parameters() ] #list of all weigths

    optimizer = torch.optim.Adam(p, lr=0.0002)
    mse = torch.nn.MSELoss()
    
    for i in range(num_iter):

        #################
        if lr_decay_epoch is not 0:
            optimizer = exp_lr_scheduler(optimizer, i, init_lr=0.0002, lr_decay_epoch=lr_decay_epoch,factor=0.8)

        #################
        def closure():
            optimizer.zero_grad()           
            x_np = net(net_input.type(dtype)).to(device)
            x_var = 2*x_np-1
            
            #y_var = x_var
            y_var = torch.matmul(A,x_var.reshape(d_image))
            
            loss = mse(y_var, y) #torch.matmul(A,x_hat)
            loss.backward()
            #mse_wrt_truth[i] = loss.data.cpu().numpy()
            
            return loss

        loss = optimizer.step(closure) 
        if i %100 == 0:
            print ('Iteration %05d   Train loss %f ' % (i, loss.item()))
        #print ('Iteration %05d   Train loss %f ' % (i, loss.detach().cpu().numpy()), '\r', end='')

    return net, net_input, loss

In [ ]:
t0 = time.time()

net, net_input, loss = fit3(net, num_channels, d_image, y=y, A=A, device= device,
                            num_iter = 12000, lr_decay_epoch=3000)
x_hat = net( net_input.type(dtype) )#.data.cpu().numpy()[0]

t1 = time.time()
grid = torchvision.utils.make_grid(x_hat, scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

#plt.imshow(x_hat.transpose(1,2,0))
plt.show()

print('\n time elapsed:' t1-t0)

save_path= 'DD'+'_'+img_name+'.png'
savefig=True
if savefig:
    plt.savefig(save_path,bbox_inches='tight')

# 5. Compressed sensing using hybrid model

## 5.1. Define the untrained network used for hybrid model

In [ ]:
#use decoder architecture or DC GAN architecture
decodetype = 'upsample' # transposeconv / upsample


if dataset == 'mnist':
    num_channels = [25,15,10] 
elif dataset== 'celeba':    
    num_channels = [120,40,20,15,10] 
else:
    num_channels = [128,64,32]
    
output_depth = img_np.shape[0] # number of output channels
net = autoencodernet(num_output_channels=output_depth,num_channels_up=num_channels,need_sigmoid=True, 
                        decodetype=decodetype
                        ).type(dtype)

print("number of parameters: ", num_param(net))
if decodetype == 'upsample':
    print(net.decoder)
elif decodetype == 'transposeconv':
    print(net.convdecoder)
net_in = copy.deepcopy(net)

In [ ]:
def fit4(G, net, num_channels, d_image, y, z_0, latentDim, num_iter = 1000, lr_decay_epoch = 0):
    G.eval()
    z = torch.normal(torch.zeros(1,latentDim)) #.to(config.device)
    z = Variable(z, requires_grad = True)
    
    
    # compute the size of (fixed) latent vector and draw it uniformly  
    totalupsample = 2**(len(num_channels)-1)
    w = np.sqrt(int(d_image/3)) # =d_image / out_channels = số chiều của mỗi cạnh ảnh
    width = int(w/(totalupsample))
    height = int(w/(totalupsample))

    shape = [1,num_channels[0], width, height]  
    print("shape of latent code B1: ", shape)

    print("initializing latent code B1...")
    net_input = Variable(torch.zeros(shape))
    net_input.data.uniform_()
    net_input.data *= 1./10

    net_input_saved = net_input.data.clone()
    noise = net_input.data.clone()
    
    # collecting all trainable parameters
    alpha_init = torch.zeros(1)
    beta_init = torch.zeros(1)
    alpha_init.data[0] = 0.5
    beta_init.data[0] = 0.5

    alpha = Variable(alpha_init, requires_grad=True)
    beta = Variable(beta_init, requires_grad=True)
    
    if decodetype=='upsample':
        p = [x for x in net.decoder.parameters() ] #list of all weigths
    elif decodetype=='transposeconv':
        p = [x for x in net.convdecoder.parameters() ] #list of all weigths
    
    
    
    #weight_decay = 0
    #optimizer = torch.optim.Adam(p, lr=0.001)
    
    optimizer = torch.optim.Adam(
    [
        {"params": p, "lr": 0.0001},
        {"params": alpha, "lr": 0.01},
        {"params": beta, "lr": 0.01},
        {"params": z, "lr": 0.1}
    ],
)
    
    mse = torch.nn.MSELoss()
    
    for i in range(num_iter):

        #################
        #if lr_decay_epoch is not 0:
        #    optimizer = exp_lr_scheduler(optimizer, i, init_lr=0.001, lr_decay_epoch=lr_decay_epoch,factor=0.8)

        #################
        def closure():
            optimizer.zero_grad()           
            alpha_clamp = alpha.clamp(0,1)
            beta_clamp = beta.clamp(0,1)
            
            x_var = alpha_clamp*G(z) + beta_clamp*(2*net(net_input.type(dtype))-1)
            #y_hat = x_hat
            
            y_var = torch.matmul(A,x_var.reshape(d_image))
            loss = mse(y_var, y) #torch.matmul(A,x_hat)
            loss.backward()
            #mse_wrt_truth[i] = loss.data.cpu().numpy()
            return loss

        loss = optimizer.step(closure) 
        if i % 100 == 0:
            print ('Iteration %04d   Train loss %f ' % (i, loss.detach().cpu().numpy()))

    return net, net_input, z, alpha, beta, loss

## 5.2. Initialization by CSGM

In [ ]:
latentDim = model.config.noiseVectorDim
G.eval()
z0, mse_wrt_truth = CSGM2(G, latentDim, y, x, A, device, num_iter=600)

#x0 = G(z0)

grid = torchvision.utils.make_grid(x0.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.axis('off')
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

save_path= 'PGGAN'+'_'+img_name+'.png'
savefig=True
if savefig:
    plt.savefig(save_path,bbox_inches='tight')

## 5.3. CS using hybrid model

In [ ]:
net, net_input, z, alpha, beta, loss = fit4(G, net, num_channels, d_image, y, z_0 = z0, 
                                            #latentDim=latentDim, num_iter = 1000, lr_decay_epoch = 0)

x_hat = alpha.clamp(0,1)*G(z) + beta.clamp(0,1)*(2*net(net_input.type(dtype)) - 1)

print(alpha,beta)

grid = torchvision.utils.make_grid(img_var, scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
#plt.imshow(x_hat_np.transpose(1,2,0))
#plt.show()
save_path= 'Hybrid'+'_'+img_name+'.png'
savefig=True
if savefig:
    plt.savefig(save_path,bbox_inches='tight') 

In [ ]:
z = torch.normal(torch.zeros(1,latentDim)) #.to(config.device)
z = Variable(z, requires_grad = True)


# compute the size of (fixed) latent vector and draw it uniformly  
totalupsample = 2**(len(num_channels)-1)
w = np.sqrt(int(d_image/3)) # =d_image / out_channels = số chiều của mỗi cạnh ảnh
width = int(w/(totalupsample))
height = int(w/(totalupsample))

shape = [1,num_channels[0], width, height]  
print("shape of latent code B1: ", shape)

print("initializing latent code B1...")
net_input = Variable(torch.zeros(shape))
net_input.data.uniform_()
net_input.data *= 1./20

net_input_saved = net_input.data.clone()
noise = net_input.data.clone()

# collecting all trainable parameters
alpha_init = torch.zeros(1)
beta_init = torch.zeros(1)
alpha_init.data[0] = 0.5
beta_init.data[0] = 0.5

alpha = Variable(alpha_init, requires_grad=True)
beta = Variable(beta_init, requires_grad=True)


if decodetype=='upsample':
    p = [x for x in net.decoder.parameters() ] #list of all weigths
elif decodetype=='transposeconv':
    p = [x for x in net.convdecoder.parameters() ] #list of all weigths



#weight_decay = 0
#optimizer = torch.optim.Adam(p, lr=0.001)

optimizer = torch.optim.Adam(
[
    {"params": p, "lr": 0.0001},
    {"params": alpha, "lr": 0.01},
    {"params": beta, "lr": 0.01},
    {"params": z, "lr": 0.1}
],
)

mse = torch.nn.MSELoss()

for i in range(1000):

    #################
    #if lr_decay_epoch is not 0:
    #    optimizer = exp_lr_scheduler(optimizer, i, init_lr=0.001, lr_decay_epoch=lr_decay_epoch,factor=0.8)

    #################
    def closure():
        optimizer.zero_grad()           
        alpha_clamp = alpha.clamp(0,1).item()
        beta_clamp = beta.clamp(0,1).item()

        x_hat = alpha_clamp*G(z) + beta_clamp*(2*net(net_input.type(dtype))-1)
        y_hat = x_hat
        #y_hat = torch.matmul(A,x_hat.reshape(d_image))
        loss = mse(x_hat, img_var) #torch.matmul(A,x_hat)
        loss.backward()
        #mse_wrt_truth[i] = loss.data.cpu().numpy()
        return loss

    loss = optimizer.step(closure) 

    print ('Iteration %05d   Train loss %f ' % (i, loss.detach().cpu().numpy()), '\r', end='')

In [ ]:
print(alpha.clamp(0,1),beta.clamp(0,1))
print(torch.min(img_var), torch.max(img_var))

In [ ]:
x_hat = alpha.clamp(0,1)*G(z) + beta.clamp(0,1)*(2*net(net_input.type(dtype)) - 1)
x_hat_np = x_hat.data.cpu().numpy()[0]
grid = torchvision.utils.make_grid(img_var, scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
#plt.imshow(x_hat_np.transpose(1,2,0))
plt.show()


In [ ]:
A = torch.randn(2,3).to(device)

In [ ]:
print(A)